# Document Classification with Azure OpenAI's GPT-4o Vision Capabilities

This sample demonstrates how to classify a document using Azure OpenAI's GPT-4o model with vision capabilities.

![Data Classification](../../../images/classification-openai.png)

This is achieved by the following process:

- Define a list of classifications, with descriptions and keywords.
- Construct a system prompt that defines the instruction for classifying document pages.
- Construct a user prompt that includes the defined classifications, and each document page as an base64 encoded image.
- Use the Azure OpenAI chat completions API with the GPT-4o model to generate a classification for each document page as a structured output.

## Objectives

By the end of this sample, you will have learned how to:

- Convert a document into a set of base64 encoded images for processing by GPT-4o.
- Use prompt engineering techniques to instruct GPT-4o to classify a document's pages into predefined categories.

## Useful Tips

- Combine this technique with a [page extraction](../extraction/README.md) approach to ensure that you extract the most relevant data from a document's pages.

## Setup

### Import modules

This sample takes advantage of the following Python dependencies:

- **pdf2image** for converting a PDF file into a set of images per page.
- **openai** to interface with the Azure OpenAI chat completions API to generate structured classification outputs using the GPT-4o model.
- **azure-identity** to securely authenticate with deployed Azure Services using Microsoft Entra ID credentials.

The following local components are also used:

- [**classification**](../modules/samples/models/classification.py) to define the classifications.
- [**accuracy_evaluator**](../modules/samples/evaluation/accuracy_evaluator.py) to evaluate the output of the classification process with expected results.
- [**openai_confidence**](../modules/samples/confidence/openai_confidence.py) to calculate the confidence of the classification process based on the `logprobs` response from the OpenAI API request.
- [**document_processing_result**](../modules/samples/models/document_processing_result.py) to store the results of the classification process as a file.
- [**stopwatch**](../modules/samples/utils/stopwatch.py) to measure the end-to-end execution time for the classification process.
- [**app_settings**](../modules/samples/app_settings.py) to access environment variables from the `.env` file.

In [1]:
import sys
sys.path.append('../modules/') # Import local modules

from IPython.display import display
import os
import pandas as pd
from dotenv import dotenv_values
import base64
import io
import json
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from concurrent.futures import ThreadPoolExecutor
from pdf2image import convert_from_bytes

from samples.app_settings import AppSettings
from samples.utils.stopwatch import Stopwatch
from samples.utils.storage_utils import create_json_file
from samples.models.document_processing_result import DataClassificationResult

from samples.models.classification import Classifications, Classification
from samples.confidence.openai_confidence import evaluate_confidence
from samples.evaluation.accuracy_evaluator import AccuracyEvaluator
from samples.evaluation.comparison import get_extraction_comparison

### Configure the Azure services

To use Azure OpenAI, the SDK is used to create a client instance using a deployed endpoint and authentication credentials.

For this sample, the credentials of the Azure CLI are used to authenticate with the deployed services.

In [2]:
# Set the working directory to the root of the repo
working_dir = os.path.abspath('../../../')
settings = AppSettings(dotenv_values(f"{working_dir}/.env"))
sample_path = f"{working_dir}/samples/python/classification/"
sample_name = "document-classification-gpt-vision"

# Configure the default credential for accessing Azure services using Azure CLI credentials
credential = DefaultAzureCredential(
    exclude_workload_identity_credential=True,
    exclude_developer_cli_credential=True,
    exclude_environment_credential=True,
    exclude_managed_identity_credential=True,
    exclude_powershell_credential=True,
    exclude_shared_token_cache_credential=True,
    exclude_interactive_browser_credential=True
)

openai_token_provider = get_bearer_token_provider(credential, 'https://cognitiveservices.azure.com/.default')

openai_client = AzureOpenAI(
    azure_endpoint=settings.openai_endpoint,
    azure_ad_token_provider=openai_token_provider,
    api_version="2024-12-01-preview" # Requires the latest API version for structured outputs.
)

### Establish the expected output

To compare the accuracy of the classification process, the expected output of the classification process has been defined in the following code block based on each page of a [Vehicle Insurance Policy](../../assets/vehicle_insurance/policy_1.pdf).

The expected output has been defined by a human evaluating the document.

> **Note**: Only the `classification` and `image_range_start` are used in the accuracy evaluation.

In [3]:
path = f"{working_dir}/samples/assets/vehicle_insurance/"
pdf_fname = "policy_1.pdf"
pdf_fpath = f"{path}{pdf_fname}"

expected = Classifications(classifications=[
    Classification(classification="Insurance Policy", image_range_start=1, image_range_end=5),
    Classification(classification="Insurance Certificate", image_range_start=6, image_range_end=6),
    Classification(classification="Terms and Conditions", image_range_start=7, image_range_end=13)
])

classification_evaluator = AccuracyEvaluator(match_keys=["classification", "image_range_start"], ignore_keys=[])

## Define classifications

The following code block defines the classifications for a document. Each classification has a name, description, and keywords that will be used to classify the document's pages.

> **Note**, the classifications have been defined based on expected content in a specific type of document, in this example, [a Vehicle Insurance Policy](../../assets/vehicle_insurance/policy_1.pdf).

In [4]:
classifications = [
    {
        "classification": "Insurance Policy",
        "description": "Specific information related to an insurance policy, such as coverage, limits, premiums, and terms, often used for reference or clarification purposes.",
        "keywords": [
            "welcome letter",
            "personal details",
            "vehicle details",
            "insured driver details",
            "policy details",
            "incident/conviction history",
            "schedule of insurance",
            "vehicle damage excesses"
        ]
    },
    {
        "classification": "Insurance Certificate",
        "description": "A document that serves as proof of insurance coverage, often required for legal, regulatory, or contractual purposes.",
        "keywords": [
            "certificate of vehicle insurance",
            "effective date of insurance",
            "entitlement to drive",
            "limitations of use"
        ]
    },
    {
        "classification": "Terms and Conditions",
        "description": "The rules, requirements, or obligations that govern an agreement or contract, often related to insurance policies, financial products, or legal documents.",
        "keywords": [
            "terms and conditions",
            "legal statements",
            "payment instructions",
            "legal obligations",
            "covered for",
            "claim settlement",
            "costs to pay",
            "legal responsibility",
            "personal accident coverage",
            "medical expense coverage",
            "personal liability coverage",
            "windscreen damage coverage",
            "uninsured motorist protection",
            "renewal instructions",
            "cancellation instructions"
        ]
    }
]

## Classify the document pages

The following code block runs the classification process using Azure OpenAI's GPT-4o model using vision capabilities.

It performs the following steps:

1. Get the document bytes from the provided file path. _Note: In this example, we are processing a local document, however, you can use any document storage location of your choice, such as Azure Blob Storage._
2. Use pdf2image to convert the document to a list of images per page as base64 strings.
3. Use Azure OpenAI's GPT-4o model and the classification definitions to provide a classification for each page of the document.

In [5]:
system_prompt = f"""You are an AI assistant that helps detect the boundaries of sub-section or sub-documents using the provided classifications.

## Classifications

{json.dumps(classifications)}
"""

In [6]:
# Prepare the user content for the OpenAI API including the classifications and the document page images.
user_content = []

In [7]:
user_text_prompt = f"""Classify insurance documents in the provided page images.

- A single classification may span multiple page images.
- A single page image may contain multiple classifications.
- If a page image does not contain a classification, ignore it."""

user_content.append({
    "type": "text",
    "text": user_text_prompt
})

In [8]:
def encode_page(page_tuple):
    idx, page = page_tuple
    byte_io = io.BytesIO()
    page.save(byte_io, format='PNG')
    base64_data = base64.b64encode(byte_io.getvalue()).decode('utf-8')
    return [
        {
            "type": "text",
            "text": f"Page {idx + 1}"
        },
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/png;base64,{base64_data}"
            }
        }
    ]

with Stopwatch() as image_stopwatch:
    with open(pdf_fpath, "rb") as f:
        document_bytes = f.read()

    pages = convert_from_bytes(document_bytes)
    
    # Process each page in parallel using multiple processes
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(encode_page, enumerate(pages)))
        user_content.extend([item for sublist in results for item in sublist])

In [9]:
with Stopwatch() as oai_stopwatch:
    completion = openai_client.beta.chat.completions.parse(
        model=settings.gpt4o_model_deployment_name,
        messages=[
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": user_content
            }
        ],
        response_format=Classifications,
        max_tokens=4096,
        temperature=0.1,
        top_p=0.1,
        logprobs=True # Enabled to determine the confidence of the response.
    )

### Understanding the Structured Outputs JSON schema

Using [Pydantic's JSON schema feature](https://docs.pydantic.dev/latest/concepts/json_schema/), the [Classification](../modules/samples/models/classification.py) data model is automatically converted to a JSON schema when applied to the `response_format` parameter of the OpenAI chat completions request.

The JSON schema is used to instruct the GPT-4o model to generate a strict output that adheres to the structure defined. The approach using Pydantic makes it easier for developers to manage the data structure in code, with helpful descriptions and examples that will be included in the final JSON schema.

Demonstrated below, you can see how the Classification data model is understood by the OpenAI request:

In [10]:
# Highlight the schema sent to the OpenAI model
print(json.dumps(Classifications.model_json_schema(), indent=2))

{
  "$defs": {
    "Classification": {
      "description": "A class representing a classification of a collection of page images from a document.",
      "properties": {
        "classification": {
          "anyOf": [
            {
              "type": "string"
            },
            {
              "type": "null"
            }
          ],
          "description": "Classification of the page, e.g., invoice, receipt, etc.",
          "title": "Classification"
        },
        "image_range_start": {
          "anyOf": [
            {
              "type": "integer"
            },
            {
              "type": "null"
            }
          ],
          "description": "If a single document associated with the classification spans multiple pages, this field specifies the start of the image range, e.g., 1.",
          "title": "Image Range Start"
        },
        "image_range_end": {
          "anyOf": [
            {
              "type": "integer"
            },
        

## Calculate the accuracy

The following code block calculates the accuracy of the classification process by comparing the actual classifications with the predicted classifications.

In [11]:
# Gets the parsed Classifications object from the completion response.
document_classifications = completion.choices[0].message.parsed

expected_dict = expected.model_dump()
classifications_dict = document_classifications.model_dump()

accuracy = classification_evaluator.evaluate(expected=expected_dict, actual=classifications_dict)

## Visualize the outputs

To provide context for the execution of the code, the following code blocks visualize the outputs of the classification process.

This includes:

- The accuracy of the classification process comparing the expected output with the output generated by Azure OpenAI's GPT-4o model.
- The confidence score of the classification process based on the log probability of the predicted classification.
- The execution time of the end-to-end process.
- The total number of tokens consumed by the GPT-4o model.
- The classification results for each page of the document.

### Understanding Accuracy vs Confidence

When using AI to classify data, both confidence and accuracy are essential for different but complementary reasons.

- **Accuracy** measures how close the AI model's output is to a ground truth or expected output. It reflects how well the model's predictions align with reality.
  - Accuracy ensures consistency in the classification process, which is crucial for downstream tasks using the data.
- **Confidence** represents the AI model's internal assessment of how certain it is about its predictions.
  - Confidence indicates that the model is certain about its predictions, which can be a useful indicator for human reviewers to step in for manual verification.

High accuracy and high confidence are ideal, but in practice, there is often a trade-off between the two. While accuracy cannot always be self-assessed, confidence scores can and should be used to prioritize manual verification of low-confidence predictions.

In [12]:
# Determines the confidence of the classifications using the log probabilities of the completion response.
confidence = evaluate_confidence(classifications_dict, completion.choices[0])

In [13]:
# Gets the total execution time of the classification process.
total_elapsed = image_stopwatch.elapsed + oai_stopwatch.elapsed

# Gets the prompt tokens and completion tokens from the completion response.
prompt_tokens = completion.usage.prompt_tokens
completion_tokens = completion.usage.completion_tokens

In [14]:
# Save the output of the data classification result.
classification_result = DataClassificationResult(
    classification=classifications_dict,
    accuracy=accuracy,
    execution_time=total_elapsed
)

create_json_file(f"{sample_path}/{sample_name}.{pdf_fname}.json", classification_result)

In [15]:
# Display the outputs of the classification process.
df = pd.DataFrame([
    {
        "Accuracy": f"{accuracy['overall'] * 100:.2f}%",
        "Confidence": f"{confidence['_overall'] * 100:.2f}%",
        "Execution Time": f"{total_elapsed:.2f} seconds",
        "Image Pre-processing Execution Time": f"{image_stopwatch.elapsed:.2f} seconds",
        "OpenAI Execution Time": f"{oai_stopwatch.elapsed:.2f} seconds",
        "Prompt Tokens": prompt_tokens,
        "Completion Tokens": completion_tokens
    }
])

display(df)
display(get_extraction_comparison(expected_dict, classifications_dict, confidence, accuracy['accuracy']))

,Accuracy,Confidence,Execution Time,Image Pre-processing Execution Time,OpenAI Execution Time,Prompt Tokens,Completion Tokens
0,100.00%,98.76%,15.69 seconds,1.83 seconds,13.86 seconds,10570,58


,Field,Expected,Extracted,Confidence,Accuracy
0,classifications_0_classification,Insurance Policy,Insurance Policy,99.94%,Match
1,classifications_0_image_range_end,5,5,89.24%,Match
2,classifications_0_image_range_start,1,1,100.00%,Match
3,classifications_1_classification,Insurance Certificate,Insurance Certificate,99.89%,Match
4,classifications_1_image_range_end,6,6,99.87%,Match
5,classifications_1_image_range_start,6,6,100.00%,Match
6,classifications_2_classification,Terms and Conditions,Terms and Conditions,100.00%,Match
7,classifications_2_image_range_end,13,13,99.92%,Match
8,classifications_2_image_range_start,7,7,99.99%,Match
